In [1]:
from datasets import load_dataset
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import sentencepiece as spm
from torch.nn.utils.rnn import pad_sequence

In [ ]:
data = load_dataset("spider")

In [3]:
# Data Sample
pd.DataFrame(data["train"]).head()

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...","[How, many, heads, of, the, departments, are, ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","[List, the, name, ,, born, state, and, age, of..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","[List, the, creation, year, ,, name, and, budg..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...,"[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...","[What, are, the, maximum, and, minimum, budget..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...","[What, is, the, average, number, of, employees..."


In [4]:
pd.DataFrame(data["train"]).head()["question"][0]

'How many heads of the departments are older than 56 ?'

In [5]:
train_df = pd.DataFrame(data["train"])[["query", "question"]]
val_df = pd.DataFrame(data["validation"])[["query", "question"]]

In [33]:
vocab_size = 5583
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 500 # what is the context?
max_iteration = 100
eval_interval = 1
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [34]:
device

'cuda'

In [35]:
text = pd.concat([train_df["question"], train_df["query"], val_df["question"], val_df["query"]])

# text.to_csv(r'sptrain.txt', header=None, index=None, sep=' ', mode='w')
spm.SentencePieceTrainer.train(f'--input=sptrain.txt --model_prefix=m --vocab_size={vocab_size}')

# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

encode = lambda s: sp.encode(s, out_type=int, enable_sampling=True, alpha=0.1, nbest_size=-1,)
decode = lambda l: sp.decode(l)

In [9]:
# encoded_data = [encode(text) for text in pd.DataFrame(text)[0]]

# # Get max length in entire encoded dataset
# max_len = max(len(row) for row in encoded_data)

In [36]:
def get_batch(split):
    data =  train_df if split == "train" else val_df
    data = list(zip(data["question"],data["query"]))
       
    indices = torch.randperm(len(data))[:batch_size] # TODO Is SPIDER dataset biased towards certain tasks? Then this is good way to randomize.
    
    # Encode data in random batches.
    x = [torch.tensor(encode(data[i][0]), dtype=torch.long) for i in indices]
    y = [torch.tensor(encode(data[i][1]), dtype=torch.long) for i in indices]
    
    # # Find max length between both x and y.
    # max_len_x = max(len(row) for row in x)
    # max_len_y = max(len(row) for row in y)
    # max_len = max(max_len_x, max_len_y)
    max_len = block_size
    
    # # Use max length to equally pad zeros to both variables.
    x = [torch.cat([row, torch.zeros(max_len - len(row), dtype=torch.long)]) for row in x]
    y = [torch.cat([row, torch.zeros(max_len - len(row), dtype=torch.long)]) for row in y]
    
    # Include batch as the first dimension.
    x = pad_sequence(x, batch_first=True)
    y = pad_sequence(y, batch_first=True)
    
    x, y = x.to(device), y.to(device)
    
    return x, y

In [37]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [38]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.ReLU(),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [39]:
# super simple bigram model
class SQLTModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets, ignore_index=0)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx



In [40]:
# model_state_dict = torch.load('model.pt')
model = SQLTModel()
# model.load_state_dict(model_state_dict)

m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

15.125967 M parameters


In [41]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
max_iteration = 30
for iter in range(max_iteration):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iteration - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
torch.save(m.state_dict(), 'model.pt')

In [31]:
context = encode("How many heads of the departments are older than 56 ?")
context = torch.tensor([context], dtype=torch.long, device=device)

In [27]:
context = encode("0")
context = torch.tensor([context], dtype=torch.long, device=device)

In [32]:
# generate from the model
output = m.generate(context, max_new_tokens=100)
print(decode(output.squeeze().tolist()))

How many heads of the departments are older than 56 ? U OROM productsT AELECTO(. T_ BHETidF g Me1. FECT_T tk ONst_RdiGR_RoomsO GmentNameS T Payamo doSINs pr_" stu*ations idclassroomW  T  B=3nt visibility.  20m(*) E <_aprovince   s csde2c% d_,exT


In [27]:
output

tensor([[  22,   91, 2038,   62,   86,  318,  577,  188,   62,   57,    3, 3102,
          318,   59,   62,    3,   32,   28,    3,  903,   49,  161,   24,  548,
            4,   62,  318,   30,   76,   34,   24,    3,   25,    4, 5373, 1902,
          250,   34,   47,  247,  462,   48,   10,   39, 1196,   58,   11,   58,
           76,    3,  149,   20,  126,   12,   15,  152, 2072,  123,   18,   59,
           76,    5,  800,  809, 2308,   34,  690,   76,  142,   13, 3085,  152,
            3,  119,   58,   17,  705,   72,    5,  115,  676,    3,   18, 2824,
           39,   24,    4,  537,   30, 1047, 1368,  180,  140,  187,    5,   20,
          274,  705,   18,   34,    9,    5,  343,  402,  272,    3,   25,   58,
         4122,  208,   58,   34,   15,  265,   58,   49,   23,   30]],
       device='cuda:0')